In [ ]:
import pandas as pd
import re
hotspot_data = pd.read_csv("hotspot_TERRA_streaming.csv")
hotspot_data = hotspot_data.sample(frac=1).reset_index(drop=True)

import pymongo
from pymongo import MongoClient
from datetime import datetime, timedelta
import random

def getNextClimateDay():
    #gets the date of the latest climate record
    client = MongoClient('192.168.0.108', 27017) #change to local ip
    db = client.fit3182_db 
    climate = db["climate"]
    hotspots = db['hotspots']
    latest_record = climate.find_one({}, sort=[('date', pymongo.DESCENDING)])
    latest_date = latest_record['date']
    client.close()
    return latest_date


latest_date = getNextClimateDay()

import pandas as pd
import random
import time
from kafka3 import KafkaProducer
import datetime as dt
from time import sleep
from json import dumps

hostip = "192.168.0.108" #change to local ip

def pushData(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully. Data: ' + str(value))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                 api_version=(0,10))
        
    except Exception as ex:
        print('could not connect to Kafka')
        print(str(ex))
    finally:
        return _producer
        
        
if __name__ == '__main__':
    topic = 'Producer3'
    
    print('starting kafka stream')
    
    producer = connect_kafka_producer()
    
    hotspot_data = hotspot_data.sample(frac=1).reset_index(drop=True)
    
    
    latest_date = latest_date + timedelta(days=1)
    dayCounter = 0
    
    for index, row in hotspot_data.iterrows():
        df = row
        df['created_on'] = latest_date.isoformat()
        df['producer'] = topic
        record_json = df.to_json()
        pushData(producer,topic, 'parsed', record_json)
        latest_date = latest_date + timedelta(minutes=288)
        sleep(2) 
    
    


starting kafka stream
Message published successfully. Data: {"latitude":-36.341,"longitude":143.636,"confidence":83.0,"surface_temperature_celcius":58.0,"created_on":"2024-11-22T00:00:00","producer":"Producer3"}
Message published successfully. Data: {"latitude":-38.0652,"longitude":142.9781,"confidence":81.0,"surface_temperature_celcius":54.0,"created_on":"2024-11-22T04:48:00","producer":"Producer3"}
Message published successfully. Data: {"latitude":-35.7073,"longitude":143.1849,"confidence":93.0,"surface_temperature_celcius":43.0,"created_on":"2024-11-22T09:36:00","producer":"Producer3"}
Message published successfully. Data: {"latitude":-38.3343,"longitude":143.8644,"confidence":59.0,"surface_temperature_celcius":43.0,"created_on":"2024-11-22T14:24:00","producer":"Producer3"}
Message published successfully. Data: {"latitude":-36.6755,"longitude":141.5039,"confidence":68.0,"surface_temperature_celcius":45.0,"created_on":"2024-11-22T19:12:00","producer":"Producer3"}
Message published su